In [ ]:
!pip install streamlit-folium==0.10.0
!pip install Pillow==9.4.0
!pip install pyproj==3.4.0
!pip install haversine==2.7.0
!pip install shapely
!pip install tqdm==4.62.3
!pip install geojson
!pip install folium
!pip install tqdm
!pip install geojson
!pip install python-dotenv==0.21.0

In [5]:
import requests
import random
import pandas as pd 
from tqdm import tqdm
import json 

import ast

# Tools

import folium
from shapely.geometry import Polygon
import numpy as np
import geojson
import matplotlib.pyplot as plt
import geopandas as gpd
import shapely.wkt
from haversine import haversine, Unit
import time
from pyproj import Geod

from shapely import wkt
from geopandas import datasets, GeoDataFrame, read_file, points_from_xy
from geopandas.tools import overlay
from geopandas.tools import sjoin

from folium.plugins import MeasureControl
from folium.plugins import MarkerCluster

import os
from dotenv import load_dotenv

load_dotenv()

True

In [8]:
YOUR_API_KEY = os.getenv('API_KEY')

In [ ]:
df_map = pd.read_csv('DATASET_FINAL.csv')

In [ ]:
df_map.head(2)

In [ ]:
df_map = df_map.drop_duplicates(subset=['Nombre_comercial','Bandera','Producto'])

In [ ]:
len(df_map)

In [ ]:
df_map['Coords'].value_counts()

In [ ]:
df_map[df_map['Coords']=="('NA', 'NA')"]['Direccion'].unique()

In [ ]:
df_map.info()

In [ ]:
df_map['Coords'][0]

In [ ]:
float(df_map['Coords'][0].replace('(','').replace(')','').split(',')[0]) # Lat

In [ ]:
float(df_map['Coords'][0].replace('(','').replace(')','').split(',')[1].strip()) # Longitud

In [ ]:
df_map['LAT'] = df_map['Coords'].apply(lambda x : float(x.replace('(','').replace(')','').split(',')[0]) if x !=  "('NA', 'NA')"  else 'Empty')
df_map['LNG'] = df_map['Coords'].apply(lambda x : float(x.replace('(','').replace(')','').split(',')[1]) if x !=  "('NA', 'NA')"  else 'Empty')

In [ ]:
df_map['LAT'].value_counts()

In [ ]:
df_map['LNG'].value_counts()

# Funciones a usar 

In [ ]:
def GetLatLon2(Address,YOUR_API_KEY): 

    url2_geocode  = f'https://geocode.search.hereapi.com/v1/geocode?q={Address}&apiKey='+YOUR_API_KEY
    
    try:
        response = requests.get(url2_geocode).json()
        CleanAddress = response['items'][0]['title'].upper()
        LAT = response['items'][0]['position']['lat']
        LON = response['items'][0]['position']['lng']
        results = [CleanAddress,round(LAT,7),round(LON,7)]
    except:
        results = ['NotFound','NA','NA']
    return results

def GetLatLon2_google(Address,YOUR_API_KEY): 

    api_url = f'https://maps.googleapis.com/maps/api/geocode/json?address={Address}&key={YOUR_API_KEY}'
    try:
        j = requests.get(api_url).json()
        CleanAddress = str(j['results'][0]['formatted_address']).upper()
        LAT = j['results'][0]['geometry']['location']['lat']
        LON = j['results'][0]['geometry']['location']['lng']
        results = [CleanAddress,round(LAT,7),round(LON,7)]
    except:
        results = ['NotFound','NA','NA']
    return results,j 


def cal_dist(geo_source,point2,unit):
    
    
    if unit == 'Km':
        distance = haversine(geo_source, point2,Unit.KILOMETERS)
    elif unit == 'm':
        distance = haversine(geo_source, point2,Unit.METERS)
    elif unit == 'miles':
        distance = haversine(geo_source, point2,Unit.MILES)
    
    return round(distance,2)


def distance_estac(geo_source,df,radio,unit):
    
    
    distancia = []
    source = []


    for i in tqdm(range(len(df)),colour = 'green'):
        distancia.append(cal_dist(geo_source,df['POINT'][i],unit))
        source.append(geo_source)

    new_df = df.copy()
    new_df['SOURCE'] = source
    new_df['DISTANCE'] = distancia
    new_df = new_df[new_df['DISTANCE']<=radio]
    new_df = new_df.reset_index()
    new_df = new_df.drop(columns ='index')
    return new_df.sort_values(by='DISTANCE',ascending=True)

def transform_df_map(df):
    

    coordenadas = []

    for i in range(len(df)):
        
        try :
            
            coord = float(df['LAT'][i]),float(df['LNG'][i])
            coordenadas.append(coord)

        except :
            
            coordenadas.append('EMPTY')
            
    df['POINT'] = coordenadas
    df = df[df['POINT']!='EMPTY']
    df = df.reset_index()
    df = df.drop(columns = 'index')
    new_df = df.copy()
    
    return new_df